This notebook demonstrates the inference of two text detoxification models in https://github.com/s-nlp/detox.

It mostly copies these two files, with adjustments for the Colab environment:
-  https://github.com/s-nlp/detox/blob/main/emnlp2021/style_transfer/condBERT/condbert_inference.ipynb
- https://github.com/s-nlp/detox/blob/main/emnlp2021/style_transfer/paraGeDi/gedi_inference.ipynb

# Getting the code and installing the requirements

In [1]:
!git clone https://github.com/s-nlp/detox

Cloning into 'detox'...
remote: Enumerating objects: 186, done.
remote: Counting objects: 100% (186/186), done.
remote: Compressing objects: 100% (164/164), done.
remote: Total 186 (delta 54), reused 118 (delta 14), pack-reused 0
Receiving objects: 100% (186/186), 17.92 MiB | 33.18 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [26]:
!pip install --upgrade transformers

  Using cached transformers-4.37.2-py3-none-any.whl (8.4 MB)
  Using cached tokenizers-0.15.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.24.0
    Uninstalling transformers-4.24.0:
      Successfully uninstalled transformers-4.24.0


In [28]:
! pip install -r detox/requirements.txt -q

# CondBERT

## Setting up

In [1]:
import os
import sys

def add_sys_path(p):
    p = os.path.abspath(p)
    print(p)
    if p not in sys.path:
        sys.path.append(p)

In [2]:
# adding the path to the condebert code root
add_sys_path('detox/emnlp2021/style_transfer/condBERT')

/u1/mdr614/FSE_majorRevision/detox-20240127T202801Z-001/detox/emnlp2021/style_transfer/condBERT


In [26]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [27]:
from importlib import reload

In [28]:
import condbert
reload(condbert)
from condbert import CondBertRewriter

In [29]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
import numpy as np
import pickle
from tqdm.auto import tqdm, trange

In [30]:
device = torch.device('cuda:0')  # please change it to e.g. 'cuda:0' if you want to use a GPU

In [31]:
vocab_root = '/u1/mdr614/FSE_majorRevision/detox-20240127T202801Z-001/detox/emnlp2021/style_transfer/condBERT/vocab/'
toxic_data_path = '/u1/mdr614/FSE_majorRevision/detox-20240127T202801Z-001/detox/emnlp2021/data/test/test_10k_toxic'

## Loading the model and the vocabularies

In [32]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)
model.to(device);

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
with open(vocab_root + "negative-words.txt", "r") as f:
    s = f.readlines()
negative_words = list(map(lambda x: x[:-1], s))
with open(vocab_root + "toxic_words.txt", "r") as f:
    ss = f.readlines()
negative_words += list(map(lambda x: x[:-1], ss))

with open(vocab_root + "positive-words.txt", "r") as f:
    s = f.readlines()
positive_words = list(map(lambda x: x[:-1], s))

In [34]:
import pickle
with open(vocab_root + 'word2coef.pkl', 'rb') as f:
    word2coef = pickle.load(f)

In [35]:
token_toxicities = []
with open(vocab_root + 'token_toxicities.txt', 'r') as f:
    for line in f.readlines():
        token_toxicities.append(float(line))
token_toxicities = np.array(token_toxicities)
token_toxicities = np.maximum(0, np.log(1/(1/token_toxicities-1)))   # log odds ratio

# discourage meaningless tokens
for tok in ['.', ',', '-']:
    token_toxicities[tokenizer.encode(tok)][1] = 3

for tok in ['you']:
    token_toxicities[tokenizer.encode(tok)][1] = 0

## Applying the model

In [36]:
reload(condbert)
from condbert import CondBertRewriter

editor = CondBertRewriter(
    model=model,
    tokenizer=tokenizer,
    device=device,
    neg_words=negative_words,
    pos_words=positive_words,
    word2coef=word2coef,
    token_toxicities=token_toxicities,
)

In [37]:
print(editor.translate('You are an idiot!', prnt=False))

you are an the !


In [38]:
editor = CondBertRewriter(
    model=model,
    tokenizer=tokenizer,
    device=device,
    neg_words=negative_words,
    pos_words=positive_words,
    word2coef=word2coef,
    token_toxicities=token_toxicities,
    predictor=None,
)

In [39]:
from multiword import masked_token_predictor_bert
reload(masked_token_predictor_bert)
from multiword.masked_token_predictor_bert import MaskedTokenPredictorBert

In [52]:
import csv
predictor = MaskedTokenPredictorBert(model, tokenizer, max_len=250, device=device, label=0, contrast_penalty=0.0)
editor.predictor = predictor

def adjust_logits(logits, label):
    return logits - editor.token_toxicities * 3

predictor.logits_postprocessor = adjust_logits

import pandas as pd

# Read the CSV file
csv_path = '/u1/mdr614/ATUC_Final/Conversion_uncivil_to_civil_random_2000/Random_2000_uncivil_comments_for_conversion/civilAlternatives_by_random2000.csv'  # Replace with your actual CSV file path
results_csv_path = '/u1/mdr614/ATUC_Final/Conversion_uncivil_to_civil_random_2000/Random_2000_uncivil_comments_for_conversion/civilAlternatives_by_CondaBert_random2000.csv' 
df = pd.read_csv(csv_path,sep="\t")

# Assuming 'uncivil' is the column containing the sentences
uncivil_sentences = df['uncivil'].tolist()

with open(results_csv_path, 'a', newline='', encoding='utf-8') as csvfile:
    # Create a CSV writer
    csv_writer = csv.writer(csvfile)

    # Process each uncivil sentence
    for input_sentence in uncivil_sentences:
        print("input: ",input_sentence) 
    
        # Assuming the rest of your code remains the same
        result = editor.replacement_loop(input_sentence, verbose=False, n_units=3)
        print()
        print("output: ",result) 
        csv_writer.writerow([input_sentence, result])

input:  eapByteBuffer[pos=0 lim=0 cap=0]]] output: ByteBufferWrapper (1798893235) [visible=[java.nio.HeapByteBuffer[pos=0 lim=33418 cap=33418]]] 2020-07-14T22:21:29.574+02:00 DEBUG [SSLConnectionContext] wrap engine: sun.security.ssl.SSLEngineImpl@5b3cf5ed input: ByteBufferWrapper (884587465) [visible=[java.nio.HeapByteBuffer[pos=0 lim=0 cap=0]]] output: ByteBufferWrapper (1798893235) [visible=[java.nio.HeapByteBuffer[pos=0 lim=33418 cap=33418]]] 2020-07-14T22:21:29.574+02:00 DEBUG [SSLConnectionContext] wrap don

output:  eapbytebuffer [ po = 0 lim = 0 cap = 0 ] ] ] output : bytebufferwrapper ( 1798893235 ) [ visible = [ java . nio . heapbytebuffer [ po = 0 lim = 33418 cap = 33418 ] ] ] 2020 - 07 - 14t22 : 21 : 29 . 574 + 02 : 00 debug [ sslconnectioncontext ] wrap engine : sun . security . ssl . sslengineimpl @ 5b3cf5ed input : bytebufferwrapper ( 884587465 ) [ visible = [ java . nio . heapbytebuffer [ po = 0 lim = 0 cap = 0 ] ] ] output : bytebufferwrapper ( 1798893235 ) [ visible =

# ParaGeDi

## Setting up

In [80]:
import torch
import numpy as np

In [81]:
add_sys_path('detox/emnlp2021/style_transfer/paraGeDi')

/home/mdr614/Downloads/detox-20240127T202801Z-001/detox/emnlp2021/style_transfer/paraGeDi


In [82]:
from importlib import reload
import gedi_adapter
reload(gedi_adapter)
from gedi_adapter import GediAdapter

In [83]:
from transformers import AutoModelForSeq2SeqLM, AutoModelForCausalLM, AutoTokenizer
t5name = 's-nlp/t5-paraphrase-paws-msrp-opinosis-paranmt'
model_path = 's-nlp/gpt2-base-gedi-detoxification'
clf_name = 's-nlp/roberta_toxicity_classifier_v1'

In [ ]:
import text_processing
reload(text_processing);

In [ ]:
import torch
#device = torch.device('cuda:0')
device = torch.device('cpu')

## Loading the model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(t5name)

Downloading:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

In [ ]:
para = AutoModelForSeq2SeqLM.from_pretrained(t5name)
para.resize_token_embeddings(len(tokenizer))

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Embedding(32100, 768)

In [ ]:
para.to(device);
para.eval();

In [ ]:
gedi_dis = AutoModelForCausalLM.from_pretrained(model_path)

Downloading:   0%|          | 0.00/871 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Some weights of the model checkpoint at s-nlp/gpt2-base-gedi-detoxification were not used when initializing GPT2LMHeadModel: ['logit_scale', 'bias']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
NEW_POS = tokenizer.encode('normal', add_special_tokens=False)[0]
NEW_NEG = tokenizer.encode('toxic', add_special_tokens=False)[0]

In [ ]:
# add gedi-specific parameters
if os.path.exists(model_path):
    w = torch.load(model_path + '/pytorch_model.bin', map_location='cpu')
    gedi_dis.bias = w['bias']
    gedi_dis.logit_scale = w['logit_scale']
    del w
else:
    gedi_dis.bias = torch.tensor([[ 0.08441592, -0.08441573]])
    gedi_dis.logit_scale = torch.tensor([[1.2701858]])
print(gedi_dis.bias, gedi_dis.logit_scale)

tensor([[ 0.0844, -0.0844]]) tensor([[1.2702]])


## Inference example

The cell below uses random sampling, so it's ok if it doesn't reproduce.

In [ ]:
%%time

dadapter = GediAdapter(model=para, gedi_model=gedi_dis, tokenizer=tokenizer, gedi_logit_coef=5, target=1, neg_code=NEW_NEG, pos_code=NEW_POS, lb=None, ub=None)
text = 'The internal policy of Trump is flawed.'
print('====BEFORE====')
print(text)
print('====AFTER=====')
inputs = tokenizer.encode(text, return_tensors='pt').to(para.device)
result = dadapter.generate(inputs, do_sample=True, num_return_sequences=3, temperature=1.0, repetition_penalty=3.0, num_beams=1)
for r in result:
    print(tokenizer.decode(r, skip_special_tokens=True))

====BEFORE====
The internal policy of Trump is flawed.
====AFTER=====
oh, goddamn it. Trump has an outdated IRL policy against Russia.
the trump you're having in his head is flawed.
the guise of the antiimmigration forking Trump is lacking in everything I
CPU times: user 12.2 s, sys: 15.8 ms, total: 12.2 s
Wall time: 12.3 s


In [ ]:
import gc

def cleanup():
    gc.collect()
    if torch.cuda.is_available() and device.type != 'cpu':
        with torch.cuda.device(device):
            torch.cuda.empty_cache()

In [ ]:
gedi_dis.to(device);
gedi_dis.bias = gedi_dis.bias.to(device)
gedi_dis.logit_scale = gedi_dis.logit_scale.to(device)
gedi_dis.eval();

In [ ]:
with open(toxic_data_path, 'r') as f:
    test_data = [line.strip() for line in f.readlines()]
print(len(test_data))

10000


In [ ]:
dadapter = GediAdapter(
    model=para, gedi_model=gedi_dis, tokenizer=tokenizer, gedi_logit_coef=10, target=0, neg_code=NEW_NEG, pos_code=NEW_POS,
    reg_alpha=3e-5, ub=0.01
)

In [ ]:
def paraphrase(text, n=None, max_length='auto', beams=2):
    texts = [text] if isinstance(text, str) else text
    texts = [text_processing.text_preprocess(t) for t in texts]
    inputs = tokenizer(texts, return_tensors='pt', padding=True)['input_ids'].to(dadapter.device)
    if max_length == 'auto':
        max_length = min(int(inputs.shape[1] * 1.1) + 4, 64)
    result = dadapter.generate(
        inputs,
        num_return_sequences=n or 1,
        do_sample=False, temperature=0.0, repetition_penalty=3.0, max_length=max_length,
        bad_words_ids=[[2]],  # unk
        num_beams=beams,
    )
    texts = [tokenizer.decode(r, skip_special_tokens=True) for r in result]
    texts = [text_processing.text_postprocess(t) for t in texts]
    if not n and isinstance(text, str):
        return texts[0]
    return texts

In [ ]:
paraphrase(test_data[:3])

["You'd be a bad guy. Oh, yeah.",
 'As snooty and overbearing as his boss.',
 'A bad society does bad things, and votes for bad politicians.']

Expected output

```
["You'd be a bad guy. Oh, yeah.",
 'As snooty and overbearing as his boss.',
 'A bad society does bad things, and votes for bad politicians.']
 ```

## Inference with reranking by classifier

In [ ]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer
clf = RobertaForSequenceClassification.from_pretrained(clf_name).to(device);
clf_tokenizer = RobertaTokenizer.from_pretrained(clf_name)

Downloading:   0%|          | 0.00/530 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at s-nlp/roberta_toxicity_classifier_v1 were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

In [ ]:
def predict_toxicity(texts):
    with torch.inference_mode():
        inputs = clf_tokenizer(texts, return_tensors='pt', padding=True).to(clf.device)
        out = torch.softmax(clf(**inputs).logits, -1)[:, 1].cpu().numpy()
    return out

In [ ]:
predict_toxicity(['hello world', 'hello aussie', 'hello fucking bitch'])

array([5.0525083e-05, 8.7885164e-05, 9.9968088e-01], dtype=float32)

In [ ]:
from gedi_adapter import GediAdapter


adapter2 = GediAdapter(
    model=para, gedi_model=gedi_dis, tokenizer=tokenizer,
    gedi_logit_coef=10,
    target=0, pos_code=NEW_POS,
    neg_code=NEW_NEG,
    reg_alpha=3e-5,
    ub=0.01,
    untouchable_tokens=[0, 1],
)


def paraphrase(text, max_length='auto', beams=5, rerank=True):
    texts = [text] if isinstance(text, str) else text
    texts = [text_processing.text_preprocess(t) for t in texts]
    inputs = tokenizer(texts, return_tensors='pt', padding=True)['input_ids'].to(adapter2.device)
    if max_length == 'auto':
        max_length = min(int(inputs.shape[1] * 1.1) + 4, 64)
    attempts = beams
    out = adapter2.generate(
        inputs,
        num_beams=beams,
        num_return_sequences=attempts,
        do_sample=False,
        temperature=1.0,
        repetition_penalty=3.0,
        max_length=max_length,
        bad_words_ids=[[2]],  # unk
        output_scores=True,
        return_dict_in_generate=True,
    )
    results = [tokenizer.decode(r, skip_special_tokens=True) for r in out.sequences]

    if rerank:
        scores = predict_toxicity(results)

    results = [text_processing.text_postprocess(t) for t in results]
    out_texts = []
    for i in range(len(texts)):
        if rerank:
            idx = scores[(i*attempts):((i+1)*attempts)].argmin()
        else:
            idx = 0
        out_texts.append(results[i*attempts+idx])
    return out_texts

torch.manual_seed(0)
paraphrase(['fuck you!', 'you are stupid!', 'you remind me of the chump .', 'he has to be a terrorist ! .'], beams=3)

["Fick 'Emmy.",
 "You'd be wrong!",
 "You'll remind me of chump?",
 'Must be a Terrorist!']

In [ ]:
batch_size = 2

In [ ]:
import os
from tqdm.auto import tqdm, trange

cleanup()

lines = test_data[:10]


for i in trange(int(len(lines) / batch_size + 1)):
    if i % 10 == 0:
        cleanup()
    t = i * batch_size
    batch = [line.strip() for line in lines[t:(t+batch_size)]]
    if not batch:
        continue

    try:
        res = paraphrase(batch, max_length='auto', beams=10)
    except RuntimeError:
        print('runtime error for batch ', i)
        try:
            cleanup()
            res = [paraphrase([text], max_length='auto', beams=3)[0] for text in batch]
        except RuntimeError:
            print('runtime error for batch ', i, 'even with batch size 1')
            res = batch
            cleanup()
    for out in res:
        print(out)

  0%|          | 0/6 [00:00<?, ?it/s]

You'd be right. You'd be right. You'd
As snooty and overbearing as its
A bad society does the wrong things, and votes for the wrong politicians.
President, he's the man. He's the man.
Colberg's a bad guy, and I'm a TSA.
Dubious, dubious, dubious, dubious.
I like you. I think you'll be an intelligent man and your contributions will be a big deal.
Luna's oh-so-stunnel, immature girl......who has no idea how to do it, which smells bad, I'd like to see
Mormonis is the wrong thing to do, it's the wrong thing to do, it's the wrong thing to do, it's the wrong thing to do, it's the right thing to do, it's the right thing to do, it's the
You'd be a bad guy, uninitiated, inconvenient thing to do.


Expected output (when running on CUDA; on CPU it migth slightly differ):
```
You'd be right. You'd be right. You'd
As snooty and overbearing as its
A bad society does the wrong things, and votes for the wrong politicians.
President, he's the man. He's the man.
Colberg's a bad guy, and I'm a TSA.
Dubious, dubious, dubious, dubious.
I like you. I think you'll be an intelligent man and your contributions will be a big deal.
Luna's oh-so-stunnel, immature girl......who has no idea how to do it, which smells bad, I'd like to see
Mormonis is the wrong thing to do. The wrong thing to do. The wrong thing to do. The wrong thing to do. The wrong thing to do. The right thing to do. The right thing to do. The right thing to do
You'd be a bad guy, uninitiated.
```